In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import csv

In [2]:
# Setup headless browser
options = Options()
options.add_argument("--headless")  # Remove this line if you want to see the browser
options.add_argument("--disable-gpu")

In [4]:
# Start browser
driver = webdriver.Chrome(options=options)

base_url = "https://uk.trustpilot.com/review/moorebarlow.com?page="

results = []

In [5]:
# Loop through pages 1 to 6
for page in range(1, 7):
    print(f"🔄 Scraping page {page}...")
    driver.get(base_url + str(page))
    time.sleep(3)  # Give time for page and JS to load

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    reviews = soup.find_all("article")

    if not reviews:
        print(f"⚠️ No reviews found on page {page}")
        continue

    for r in reviews:
        name_block = r.find("span", attrs={"data-consumer-name-typography": "true"})
        customer_name = name_block.get_text(strip=True) if name_block else ""

        text_block = r.find("p", attrs={"data-service-review-text-typography": "true"})
        review_text = text_block.get_text(strip=True) if text_block else ""

        rating_block = r.find("div", attrs={"data-service-review-rating": True})
        rating = rating_block["data-service-review-rating"] if rating_block else ""

        date_block = r.find("time")
        review_date = date_block["datetime"] if date_block else ""

        results.append([customer_name, rating, review_text, review_date])

driver.quit()

🔄 Scraping page 1...
🔄 Scraping page 2...
🔄 Scraping page 3...
🔄 Scraping page 4...
🔄 Scraping page 5...
🔄 Scraping page 6...


In [6]:
# Save to CSV
with open("trustpilot_reviews.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Customer Name", "Rating", "Review Text", "Date"])
    writer.writerows(results)

print(f"✅ Done! {len(results)} reviews saved to trustpilot_reviews.csv")

✅ Done! 129 reviews saved to trustpilot_reviews.csv
